Solving Linear Regression with Gradient Descent using Tensorflow 2.0

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn import preprocessing

/anaconda3/envs/deep_learning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/anaconda3/envs/deep_learning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/envs/deep_learning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/envs/deep_learning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/anaconda3/envs/deep_learning/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, m

In [18]:
raw_csv_data = np.loadtxt('/Users/sayanbi/Desktop/Stuffs/Audiobooks-data.csv', delimiter=',')
unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1]

In [19]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

In [9]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [11]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [12]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [15]:
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

In [16]:
np.savez('audiobook_data_train', inputs=train_inputs, targets=train_targets)
np.savez('audiobook_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('audiobook_data_test', inputs=test_inputs, targets=test_targets)

In [20]:
npz = np.load('audiobook_data_train.npz')
train_inputs, train_targets  = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('audiobook_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('audiobook_data_test.npz')
test_inputs, test_targets  = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [21]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [22]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [34]:
BATCH_SIZE = 100
NUM_EPOCHS = 100
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(patience=2)

Load the Data

In [35]:
model.fit(train_inputs, 
          train_targets, 
          epochs=NUM_EPOCHS, 
          validation_data=(validation_inputs, validation_targets), 
          callbacks=[EARLY_STOPPING],
          verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.2051 - accuracy: 0.9234 - val_loss: 0.2679 - val_accuracy: 0.8993
Epoch 2/100
3579/3579 - 0s - loss: 0.2039 - accuracy: 0.9220 - val_loss: 0.2707 - val_accuracy: 0.9038
Epoch 3/100
3579/3579 - 0s - loss: 0.2053 - accuracy: 0.9226 - val_loss: 0.2623 - val_accuracy: 0.8926
Epoch 4/100
3579/3579 - 0s - loss: 0.2034 - accuracy: 0.9215 - val_loss: 0.2551 - val_accuracy: 0.8949
Epoch 5/100
3579/3579 - 0s - loss: 0.2083 - accuracy: 0.9215 - val_loss: 0.2759 - val_accuracy: 0.9016
Epoch 6/100
3579/3579 - 0s - loss: 0.2018 - accuracy: 0.9246 - val_loss: 0.2722 - val_accuracy: 0.8993


In [36]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 27us/sample - loss: 0.1960 - accuracy: 0.9375


In [37]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.20. Test accuracy: 93.75%
